#Investigando la API

Primero realicé las llamadas sumando un milisegundo a la última entry fetcheada de la api call previa, pero luego me preocupé. Qué pasa si me retorna 100 entries, pero la última timestamp tiene entries que no caben en esas 100? Si le sumo un milisegundo, habrían algunas que quedarían fuera, y no habría manera de fetchearlas? Qué pasa si en un milisegundo en específico, hay más de 100 entries? En ese caso, sería imposible fetchearlas todas, dado que se obtienen desde el tiempo más reciente, y no hay paginación

Por lo que este script es para contar cuántas entries hay por timestamp

In [41]:
import time
import requests
from collections import defaultdict
from datetime import datetime, timezone, timedelta

market_id = 'btc-clp'
url = f'https://www.buda.com/api/v2/markets/{market_id}/trades'

end_time = datetime(2024, 3, 1, 12, 00, 0, tzinfo=timezone(timedelta(hours=-3)))
end_timestamp = int(end_time.timestamp() * 1000)

all_timestamps = []
timestamp_counts = defaultdict(int)
limit = 100

response = requests.get(url, params={
    'timestamp': end_timestamp,
    'limit': limit
})
data = response.json()

# collect timestamps
for entry in data['trades']['entries']:
    trade_timestamp = entry[0]
    all_timestamps.append(trade_timestamp)
    timestamp_counts[trade_timestamp] += 1

total_entries_fetched = len(all_timestamps)

print("Timestamp counts:")
for timestamp, count in timestamp_counts.items():
    print(f"{timestamp}: {count} entries")

print("\nSummary:")
print(f"Total API calls made: {calls_made}")
print(f"Total trades fetched: {total_entries_fetched}")


Timestamp counts:
1709305153922: 2 entries
1709305151112: 1 entries
1709305128982: 1 entries
1709305018005: 1 entries
1709304977159: 1 entries
1709304931292: 2 entries
1709304905888: 1 entries
1709304780072: 2 entries
1709304759582: 1 entries
1709304698602: 1 entries
1709304688274: 4 entries
1709304586457: 1 entries
1709304554484: 2 entries
1709304530350: 1 entries
1709304509894: 1 entries
1709304413060: 1 entries
1709304342767: 2 entries
1709304316860: 2 entries
1709304255873: 1 entries
1709304174872: 1 entries
1709304159387: 1 entries
1709304088447: 3 entries
1709304068260: 1 entries
1709304037636: 1 entries
1709304037508: 1 entries
1709304032066: 4 entries
1709304021541: 1 entries
1709303995481: 1 entries
1709303931410: 1 entries
1709303925964: 1 entries
1709303879589: 1 entries
1709303869312: 1 entries
1709303818567: 1 entries
1709303808126: 1 entries
1709303777786: 1 entries
1709303635825: 1 entries
1709303509058: 1 entries
1709303503697: 1 entries
1709303396541: 1 entries
1709303

Esto me dice

*   Efectivamente no hay solo una entry por timestamp, el sistema no va encolando los registros con una queue alterando los timestamps reales
*   El límite de 100 no es estricto, por lo que si no se ha terminado de fetchear para una timestamp, se continúa hasta terminar al parecer
*   En conclusión simplemente ir sumando un milisegundo a las llamadas hasta barrer el rango requerido tiene sentido
*   quedo tranquila



#¿Cuánto dinero (en CLP) se transó durante el evento "Black Buda" BTC-CLP ?

In [77]:
import requests
import time
from datetime import datetime

# recibe el id del mercado, los límites del rango de tiempo
#retorna todas las trades en ese período
def get_trades(market_id, start_timestamp, end_timestamp):
    trades = []
    url = f'https://www.buda.com/api/v2/markets/{market_id}/trades'
    params = {'timestamp': end_timestamp, 'limit': 100}

    while True:
        response = requests.get(url, params=params)
        data = response.json()
        entries = data['trades']['entries']
        for entry in entries:
            trade_time = int(entry[0])
            if start_timestamp <= trade_time <= end_timestamp:
                trades.append(entry)

        if not entries or int(entries[-1][0]) < start_timestamp:
            break

        least_recent_timestamp = int(entries[-1][0])
        params['timestamp'] = least_recent_timestamp -1

    return trades



In [73]:

market_id = 'btc-clp'
start_time = datetime(2024, 3, 1, 12, 0, 0)  # 12:00 GMT-3
end_time = datetime(2024, 3, 1, 13, 0, 0)  # 13:00 GMT-3

start_timestamp = int(start_time.timestamp() * 1000)
end_timestamp = int(end_time.timestamp() * 1000)


# Obtener las transacciones
trades = get_trades(market_id, start_timestamp, end_timestamp)

# Calcular el total de dinero transado en CLP
total_value_clp = 0.0

for trade in trades:
    amount = float(trade[1])
    price = float(trade[2])
    total_value_clp += amount * price

# Truncar el resultado a 2 decimales
total_value_clp = int(total_value_clp * 100) / 100.0

print(f"Total de dinero transado durante el evento 'Black Buda' en CLP: {total_value_clp}")


61
Total de dinero transado durante el evento 'Black Buda' en CLP: 35583121.0


#En comparación con el mismo día del año anterior, ¿cuál fue el aumento porcentual en el volumen de transacciones (en BTC)? (truncar en 2 decimales)

In [97]:
def percentual_difference(value1, value2):
    return (((value2 - value1)/value1) * 100)

In [99]:
#2024
total_value_btc_2024 = 0.00

for trade in trades:
    amount = float(trade[1])
    total_value_btc_2024 += amount
total_value_btc_2024 = int(total_value_btc_2024 * 100) / 100.0

print(f"total transado en btc en 2024 en periodo black buda: {total_value_btc_2024}")
print(f"cantidad de transacciones en 2024 en periodo black buda: {len(trades)}")

#2023

start_time_2023 = datetime(2023, 3, 1, 12, 0, 0)  # 12:00 GMT-3
end_time_2023 = datetime(2023, 3, 1, 13, 0, 0)  # 13:00 GMT-3

start_timestamp_2023 = int(start_time_2023.timestamp() * 1000)
end_timestamp_2023 = int(end_time_2023.timestamp() * 1000)

total_value_btc_2023 = 0.00
trades_2023 = get_trades(market_id, start_timestamp_2023, end_timestamp_2023)

for trade in trades_2023:
    amount = float(trade[1])
    total_value_btc_2023 += amount
total_value_btc_2023 = int(total_value_btc_2023 * 100) / 100.0

print(f"total transado en btc en 2023 en periodo black buda: {total_value_btc_2023}")
print(f"cantidad de transacciones en 2023 en periodo black buda: {len(trades_2023)}")

print('---')

print(f"DIFERENCIA PORCENTUAL de cantidad de BTC TRANSADO en periodo black buda: {percentual_difference(total_value_btc_2023, total_value_btc_2024)}")
print(f"DIFERENCIA PORCENTUAL de cantidad de TRANSACCIONES en periodo black buda: {percentual_difference(len(trades_2023), len(trades))}")


total transado en btc en 2024 en periodo black buda: 0.59
cantidad de transacciones en 2024 en periodo black buda: 61
total transado en btc en 2023 en periodo black buda: 0.65
cantidad de transacciones en 2023 en periodo black buda: 36
---
DIFERENCIA PORCENTUAL de cantidad de BTC TRANSADO en periodo black buda: -9.230769230769239
DIFERENCIA PORCENTUAL de cantidad de TRANSACCIONES en periodo black buda: 69.44444444444444


Se transó bastante más btc el año pasado, pero en menos transacciones. quizás estaba más barato

#Considerando que la comisión normal corresponde a un 0.8% ¿Cuánto dinero (en CLP) se dejó de ganar debido a la liberación de comisiones durante el BlackBuda?

In [102]:
total_value_clp*0.008

284664.968